# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
#MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
system_prompt = "You are an assistant that takes a technical question, and responds with an explanation"

In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [10]:
messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ]

In [11]:
def stream_brochure(system_prompt, question, MODEL, messages):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [12]:
stream_brochure(system_prompt, question, MODEL_GPT, messages)

This line of code is using Python's `yield from` construct along with a set comprehension to extract the authors of books from a collection called `books`. Let’s break it down:

1. **`{book.get("author") for book in books if book.get("author")}`**: This is a set comprehension, which creates a set of unique authors from the `books` collection. Here’s how it works:
   - `for book in books`: This iterates over each `book` in the `books` collection.
   - `book.get("author")`: This attempts to retrieve the value associated with the key `"author"` from the `book` dictionary.
   - `if book.get("author")`: This condition checks if the author exists (i.e., it is not `None` or an empty string). If it exists, it will be included in the resulting set.

   The result of this comprehension is a set of unique authors (since sets do not allow duplicate values).

2. **`yield from`**: This is a keyword used in generator functions in Python. It yields each item from the iterable that follows it, one at a time. In this case, it will yield each author from the set created by the comprehension.

### Practical Scenario:
Imagine you have a list of books with potential duplicates in authors, and you want to create a generator that yields each unique author only if they are listed for at least one book. The line of code effectively provides a way to do this.

### Why Use This?
- **Uniqueness**: By using a set comprehension, you ensure that each author is only yielded once, regardless of how many books they are associated with.
- **Lazy Evaluation**: The use of `yield from` allows for lazy evaluation, meaning that authors are generated only as needed instead of creating a complete list in memory upfront.
- **Conciseness**: This one-liner is a concise way of expressing what could take multiple lines: filtering, deduplicating, and yielding.

In sum, this line of code efficiently extracts and yields unique authors from a list of books, ensuring that only existing authors are included, while also promoting memory efficiency through lazy evaluation.

In [13]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)

print(response.choices[0].message.content)

**Understanding the Yield From Statement**

The given code snippet is written in Python 3.x and utilizes a feature called "yield from". Here's a breakdown of what it does:

**Individual Components:**

1. `{... for book in books}` creates a generator expression, iterating over each element (`book`) in the `books` collection.
2. `if book.get("author")` filters the results to only include elements where `book.get("author")` returns a truthy value (i.e., not None, empty string, etc.)
3. `{...}` applies another generator expression around the previous expressions
4. `yield from ...` is the "yield from" statement, which takes all items from the inner iterable and yields them one by one.

**Putting It Together:**

When you put it all together, this code:

1. Filters a collection of books (`books`) to only include those with a valid author information.
2. For each book that passes the filter, it yields the author's name as if they were generating values directly from the outer scope.

In essen